# **Visualiser des images: partie 2**

## Introduction à l'analyse des images - 32M7132

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

<div class="alert alert-block alert-info">
<b>Visualiser des images partie 2</b> : jusqu'ici, nous avons vu l'importance des métadonnées qui sont liées à l'image. Ces métadonnées permettent d'affiner nos visualisations. IVPY est un outil, mis au point par Damon Crockett à l'université de Yale, qui utilise les métadonnées pour proposer des visualisations.
</div>

## **Plan du cours**

> **À partir des métadonnées**
> * Importer un CSV
> * Extraire des métadonnées des images
> * Joindre les métadonnées de plusieurs CSV

> **Du CSV à IVPY**
> * Choisir notre CSV 
> * Lier les métadonnées aux images
> * Lier les métadonnées à IVPY

> **Visualiser avec IVPY**
> * Montage, histogram, scatter, compose
> * Enregistrer les images

<div class="alert alert-block alert-danger">
<b>Attention: </b> avant d'utiliser le Notebook, il faut le déplacer dans un clone du repo IVPY (ivpy/src).
</div>

In [ ]:
# librairies 

import pandas as pd
from ivpy import attach,show,compose,montage,histogram,scatter
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

## **À partir des métadonnées**

Les visualisations avec IVPY intègrent les métadonnées.Cela signifie qu'en amont il nous faut dégager des métadonnées autour de nos images. Lors des derniers cours nous avons vu comment le faire. Maintenant, il s'agit de tout mettre en commun et créer un CSV unique.

### Importer un CSV

Reprenez votre CSV de base, celui qui est lié aux images fauves que nous traitons depuis quelques semaines.

In [ ]:
# importer le csv

df = pd.read_csv("../../32M7132-TP_images2D_2022/fauvisme1904_1908.csv", index_col=[0])

In [ ]:
df.head()

### Extraire des métadonnées des images

Pour visualiser les images, on extrait la moyenne des compostantes suivantes: teinte, saturation, valeur (HSV en anglais). Afin de pouvoir ajouter les données à notre CSV, il faut suivre ces différentes étapes:

> * importer les images
> * extraire les composantes et leur moyennes
> * inscrire les données dans un tableau

#### Importer les images

In [ ]:
# définir le chemin qui mène au dossier

path = "../../32M7132-TP_images2D_2022/fauvisme_1904_1908"
os.path.isdir(path)

In [ ]:
# importer les images à partir de la dataframe

images=[]
for index, row in df.iterrows():    
    
    path_img = os.path.join(path, row.Filename)
    image = cv2.imread(path_img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)

In [ ]:
# Combien d'images importées?

len(images)

#### Extraire les composantes et leur moyenne

In [ ]:
# transformer une image en référentiel HSV 

img = cv2.cvtColor(images[0], cv2.COLOR_BGR2HSV)
img

In [ ]:
# extraire les moyennes

h = img[:,:,0].mean()
s = img[:,:,1].mean()
v = img[:,:,2].mean()

print(f"Moyenne teinte: {h} \nMoyenne saturation : {s} \nMoyenne valeur : {v}")

In [ ]:
# On peut automatiser tout ça depuis l'importation des images et enregistrer dans un dictionnaire
# Créons les colonnes dans 

df["teinte"] = ""
df["saturation"] = ""
df["valeur"] = ""

df.head()

In [ ]:
# Extraire et ajouter les données

for index, row in df.iterrows():    
    
    path_img = os.path.join(path, row.Filename)
    image = cv2.imread(path_img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h = image[:,:,0].mean()
    df.at[index, "teinte"] = h
    s = image[:,:,1].mean()
    df.at[index, "saturation"] = s
    v = image[:,:,2].mean()
    df.at[index, "valeur"] = v

In [ ]:
df.head()

### Joindre les métadonnées de plusieurs CSV

On peut également ajouter les données que vous avez extraites manuellement

In [ ]:
# Importer votre csv

df2 = pd.read_csv("", index_col=[0])

In [ ]:
# Joindre selon le nom du fichier

merged = df.join(df2.set_index(['Filename']), on = ['Filename'], how = 'right', lsuffix='_x')
merged
#merged.to_csv("fauvisme_hsv.csv")

## **Du CSV à IVPY**

Pour que IVPY puisse créer les visualisation quelques opérations préparatoires sont nécessaires

### Choisir notre CSV 

In [ ]:
# On reprendre notre CSV ou on importe un autre

df = pd.read_csv("fauvisme_hsv.csv", index_col=[0])
df

### Lier les métadonnées aux images

Il faut d'abord lier le tableau de métadonnées aux images: pour ce faire on va créer une nouvelle colonne avec le chemin

In [ ]:
# on reprend notre chemin
path

In [ ]:
df.Filename = [path+"/"+item for item in df.Filename.astype(str)]
df = df.drop_duplicates()
df.head()

In [ ]:
# on vérifie le nom

df.iloc[0].Filename

In [ ]:
# Verification pour l'image

image = cv2.imread(df.iloc[0].Filename)
print(f"Path to images is correct : {type(image) == np.ndarray}")

### Lier les métadonnées à IVPY


Pour utiliser IVPY, il faut simplement indiquer le chemin dans le tableau pour lire les images. On peut également choisir le fond sur lequel les images seront visualisées.

In [ ]:
# On remet l'index à zéro

df = df.reset_index()

In [ ]:
attach(df,'Filename')

In [ ]:
# est-ce que ça marche ?

show(pathcol=0) #or show(1)

In [ ]:
# RGB triplet

R = 39 #red
G = 33 #green
B = 122 #blue

# couleurs utilisées
color = (R,G,B)

# visualiser
c = np.array([[[R,G,B],
               [R,G,B],
               [R,G,B]],
              [[R,G,B],
               [R,G,B],
               [R,G,B]]])

print(color)
plt.imshow(c)
plt.show()

## **Visualiser avec IVPY**

IVPY a plusieurs manières de visualiser les images. Voilà les princiaples fonctions

>Montage
>>Viusaliser les images en cercle ou en en carré, avec des arguments

>Histogram
>>Crée un histogramme selon les colonnes indiquées

>Scatter
>>Dispose les images selon des points répartis le long de deux axes

>Compose
>>Permet de concatener plusieurs graphiques

>Save
>>Sauvegarder les visualisations

Et quelques paramètres:

>bg
>>Choix de la couleur en arrière plan (défninie avant)

>thumb
>>Taille de l'image: plus la taille est grande plus l'image sera de qualité (enregistrer directement l'image pour éviter les problèmes)

>xcol/ycol
>>Classification selon les colonnes

>facetcol
>>Classifie au sein du même graphique

>notecol
>>Annotation sur les images

### Montage, histogram, scatter, compose

Voyons les différentes fonctions et leurs arguments

In [ ]:
# Montage

montage(thumb=50,shape='square', bg=color, xcol="saturation", ascending=True)

In [ ]:
# Reprenons Sonia Delaunay

sub_df = df[df.Artist=='Sonia Delaunay']
sub_df

In [ ]:
montage(pathcol=sub_df.Filename, 
        shape='square',
        xcol=sub_df.Date,
        notecol= sub_df.Artist + ", " +sub_df.Artwork + ", "+ sub_df.Date.astype(str),
        thumb=450, 
        bg=color)


In [ ]:
# Histogram

histogram(xcol=df.Date, thumb=80,ycol='valeur',bins=5, bg=color)

In [ ]:
histogram(xcol=df.teinte,thumb=180,ycol='saturation',coordinates='polar',bins=40)

In [ ]:
sub_df_date = df[(df['Date'] == 1904)]
sub_df_date

In [ ]:
histogram(pathcol=sub_df_date.Filename, xcol=sub_df_date.valeur,notecol= sub_df_date.Artist , 
          thumb=80,ycol=sub_df_date.saturation,bins=10, bg=color)

In [ ]:
# Scatter

scatter('teinte','saturation', side = 400, bg=color, thumb=20)

In [ ]:
# Compose

sub_df_1904 = df[(df['Date'] == 1904)]
sub_df_1905 = df[(df['Date'] == 1905)]
sub_df_1906 = df[(df['Date'] == 1906)]
sub_df_1907 = df[(df['Date'] == 1907)]
sub_df_1908 = df[(df['Date'] == 1908)]


h1904 = histogram(pathcol=sub_df_1904.Filename, xcol=sub_df_1904.valeur,notecol= sub_df_1904.Artist , 
          thumb=80,ycol=sub_df_1904.saturation,bins=10, bg=color)
h1905 = histogram(pathcol=sub_df_1905.Filename, xcol=sub_df_1905.valeur,notecol= sub_df_1905.Artist , 
          thumb=80,ycol=sub_df_1905.saturation,bins=10, bg=color)
h1906 = histogram(pathcol=sub_df_1906.Filename, xcol=sub_df_1906.valeur,notecol= sub_df_1906.Artist , 
          thumb=80,ycol=sub_df_1906.saturation,bins=10, bg=color)
h1907 = histogram(pathcol=sub_df_1907.Filename, xcol=sub_df_1907.valeur,notecol= sub_df_1907.Artist , 
          thumb=80,ycol=sub_df_1907.saturation,bins=10, bg=color)
h1908 = histogram(pathcol=sub_df_1908.Filename, xcol=sub_df_1908.valeur,notecol= sub_df_1908.Artist , 
          thumb=80,ycol=sub_df_1908.saturation,bins=10, bg=color)

compose(h1904, h1905, h1906, h1907, h1908)

### Enregistrer les images

In [ ]:
c = compose(h1904, h1905, h1906, h1907, h1908)
c.save("compose.png")